In [1]:
import numpy as np 
import pandas as pd

import math
import sys
import os

import gymnasium as gym
gym.__version__

from citylearn.citylearn import CityLearnEnv
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper
from citylearn.reward_function import RewardFunction

sys.path.append("../custom_reward")
from custom_reward.custom_reward import CustomReward

from stable_baselines3 import SAC

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
class WrapperEnv:
    """
    Env to wrap provide Citylearn Env data without providing full env
    Preventing attribute access outside of the available functions
    """
    def __init__(self, env_data):
        self.observation_names = env_data['observation_names']
        self.action_names = env_data['action_names']
        self.observation_space = env_data['observation_space']
        self.action_space = env_data['action_space']
        self.time_steps = env_data['time_steps']
        self.seconds_per_time_step = env_data['seconds_per_time_step']
        self.random_seed = env_data['random_seed']
        self.buildings_metadata = env_data['buildings_metadata']
        self.episode_tracker = env_data['episode_tracker']
    
    def get_metadata(self):
        return {'buildings': self.buildings_metadata}

In [3]:
def makeEnv(schema_path, reward_function):
    # create environment
    env = CityLearnEnv(schema = schema_path, reward_function = reward_function, central_agent=False)

    env_data = dict(
        observation_names = env.observation_names,
        action_names = env.action_names,
        observation_space = env.observation_space,
        action_space = env.action_space,
        time_steps = env.time_steps,
        random_seed = None,
        episode_tracker = None,
        seconds_per_time_step = None,
        buildings_metadata = env.get_metadata()['buildings']
    )

    wrapper_env = WrapperEnv(env_data)
    return env, wrapper_env


In [4]:
schema_path = "data/schema.json"

env, wrapper_env = makeEnv(schema_path, CustomReward)

# wrap environment for use in stablebaselines3
# env = NormalizedObservationWrapper(env)
# env = StableBaselines3Wrapper(env)

In [5]:
obs = env.reset()
# display(env.observation_space[0].sample)
sum([obs.shape[0] for idx, obs in enumerate(env.observation_space)])
[obs.shape for obs in env.observation_space]
[act.shape for act in env.action_space]
# display(env.action_space)
# print(np.array(env.reset()).shape)

[(3,), (3,), (3,)]

In [6]:
# wrapper_env.action_names
np.array(env.get_metadata())

array({'uid': '8f772e8e457b49a0a9acc118ed44c8b8', 'random_seed': 16590566, 'simulation_time_steps': 720, 'seconds_per_time_step': 3600.0, 'reward_function': 'CustomReward', 'central_agent': False, 'shared_observations': ['month', 'day_type', 'hour', 'daylight_savings_status', 'outdoor_dry_bulb_temperature', 'outdoor_dry_bulb_temperature_predicted_6h', 'outdoor_dry_bulb_temperature_predicted_12h', 'outdoor_dry_bulb_temperature_predicted_24h', 'outdoor_relative_humidity', 'outdoor_relative_humidity_predicted_6h', 'outdoor_relative_humidity_predicted_12h', 'outdoor_relative_humidity_predicted_24h', 'diffuse_solar_irradiance', 'diffuse_solar_irradiance_predicted_6h', 'diffuse_solar_irradiance_predicted_12h', 'diffuse_solar_irradiance_predicted_24h', 'direct_solar_irradiance', 'direct_solar_irradiance_predicted_6h', 'direct_solar_irradiance_predicted_12h', 'direct_solar_irradiance_predicted_24h', 'carbon_intensity', 'electricity_pricing', 'electricity_pricing_predicted_6h', 'electricity_pri

In [7]:
from custom_agent.custom_SAC.ma_sac_agents import Agents
Agents(env)


/home/hkolstee/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
